In [263]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from http import HTTPStatus
from pprint import pprint as pp
from datetime import datetime, date
import re

In [247]:
mlb_today_url = r'https://www.espn.com/mlb/schedule'

s = requests.Session()
r = s.get(mlb_today_url, timeout=10)

print(f"Connection status: {HTTPStatus(r.status_code).phrase}")

Connection status: OK


In [248]:
soup = BeautifulSoup(r.text, 'lxml')

matchups = []

for day in soup.findAll("div", {"class":"ScheduleTables mb5 ScheduleTables--mlb ScheduleTables--baseball"}):
    try:
        dt = datetime.strptime(day.find("div", {"class":"Table__Title"}).text.strip(), "%A, %B %d, %Y").date()
    except:
        dt = pd.NaT

    for match in day.findAll("td", {"class":["date__col Table__TD","teams__col Table__TD"]}):
        try:
            url_find = match.find("a")['href']
            if bool(re.search(r"/mlb/game", url_find)):
                game_url = r"https://www.espn.com" + url_find
                game_row = (dt, game_url)
                matchups.append(game_row)
            else:
                continue
        except:
            continue
    
matchup_df = pd.DataFrame(matchups, columns=['MATCH_DATE','GAME_URL'])

matchup_df.head()

,MATCH_DATE,GAME_URL
0,2022-05-18,https://www.espn.com/mlb/game?gameId=401354799
1,2022-05-18,https://www.espn.com/mlb/game?gameId=401354793
2,2022-05-18,https://www.espn.com/mlb/game?gameId=401354796
3,2022-05-18,https://www.espn.com/mlb/game?gameId=401354798
4,2022-05-18,https://www.espn.com/mlb/game?gameId=401354806


In [264]:
def get_pitchers(url):
    try:
        s = requests.Session()
        r = s.get(url, timeout=10)

        soup = BeautifulSoup(r.text, 'lxml')

        starting_pitchers = []

        for pitchers in soup.findAll("table", {"data-type":"pitching"}):
            pitcher = pitchers.a['href']
            starting_pitchers.append(pitcher)

        if len(starting_pitchers) == 0:
            away_pitcher = np.nan
            home_pitcher = np.nan
        else:
            away_pitcher = starting_pitchers[0]
            home_pitcher = starting_pitchers[1]

        return (away_pitcher, home_pitcher)

    except:
        return (None, None)

In [267]:
matchup_df['AWAY_PITCHER'], matchup_df['HOME_PITCHER'] = zip(*matchup_df['GAME_URL'].map(get_pitchers))


In [268]:
matchup_df.head()


,MATCH_DATE,GAME_URL,AWAY_PITCHER,HOME_PITCHER
0,2022-05-18,https://www.espn.com/mlb/game?gameId=401354799,https://www.espn.com/mlb/player/_/id/32082,https://www.espn.com/mlb/player/_/id/39434
1,2022-05-18,https://www.espn.com/mlb/game?gameId=401354793,https://www.espn.com/mlb/player/_/id/33829,https://www.espn.com/mlb/player/_/id/39251
2,2022-05-18,https://www.espn.com/mlb/game?gameId=401354796,https://www.espn.com/mlb/player/_/id/34973,https://www.espn.com/mlb/player/_/id/39875
3,2022-05-18,https://www.espn.com/mlb/game?gameId=401354798,https://www.espn.com/mlb/player/_/id/4684365,https://www.espn.com/mlb/player/_/id/36071
4,2022-05-18,https://www.espn.com/mlb/game?gameId=401354806,https://www.espn.com/mlb/player/_/id/41165,https://www.espn.com/mlb/player/_/id/39671
